In [ ]:
import pandas as pd
import numpy as np
import utils.geohash_utils as ghu
from easydict import EasyDict
import utils.data_preprocess_utils as dpu
cfg = EasyDict(dpu.get_config('./cfg/network_extract_cfg.yaml'))

In [ ]:
df = pd.read_feather(cfg.wp_dir)

In [ ]:
precision_max = cfg.precision_max
bounds = cfg.bounds
tree = ghu.QuadTree(bounds, precision_max)
tree.init_subgrids(tree.root)
tree.init_ship_dic_df(df)
tree.init_ship_cnt()
tree.filter_nodes(1)
leaf_nodes = tree.get_leaf_nodes()
leaf_nodes = ghu.assign_label(leaf_nodes)
filtered_grids_map = ghu.visualize_node_list(df, leaf_nodes, color='red')
waypoints = ghu.leaf_nodes2waypoints(leaf_nodes)
convex_hulls = ghu.get_convex_hulls(waypoints)
convex_hulls.to_feather(cfg.convex_hull_save_dir)

In [ ]:
data_dir = cfg.data_dir
files = dpu.get_all_files(data_dir)
def get_subset(files_path, number):
    idx = np.random.choice(len(files_path), number, replace=False)
    # return a concated dataframe
    return pd.concat([pd.read_feather(files_path[i]) for i in idx], ignore_index=True)
df = get_subset(files, 25)
df = df.sort_values(by=['MMSI', 'time']).reset_index(drop=True)
adjacency_matrix, length_matrix, width_matrix = ghu.get_adjacency_shape_matrix(convex_hulls, df)
edge_list, adjacency_matrix, distance_matrix = ghu.get_edges_list(convex_hulls, adjacency_matrix, flow_threshold=6, dis_threshold=34)
np.save(cfg.adjacency_matrix_save_dir, adjacency_matrix)
np.save(cfg.distance_matrix_save_dir, distance_matrix)
np.save(cfg.length_matrix_save_dir, length_matrix)
np.save(cfg.width_matrix_save_dir, width_matrix)


In [ ]:
adjacency_matrix_bk = adjacency_matrix.copy()

In [ ]:
adjacency_matrix = adjacency_matrix_bk.copy()

In [ ]:
LAT_MAX, LAT_MIN, LON_MAX, LON_MIN = (53.9884, 9.7614, 56.4261, 15.1117)
import folium
import geopandas as gpd
from shapely.geometry import Polygon
from folium import Popup
cfg = EasyDict(dpu.get_config('./cfg/network_extract_cfg.yaml'))
gdf = gpd.read_feather(cfg.convex_hull_save_dir)
waypoints = np.array(gdf['centroid'].apply(lambda p: (p.y, p.x)).to_list())
m = folium.Map(location=[(LAT_MAX + LAT_MIN) / 2, (LON_MAX + LON_MIN) / 2], zoom_start=8, tiles='OpenStreetMap')
connection_cnt = np.zeros((waypoints.shape[0],))
for i in range(waypoints.shape[0]):

    folium.Circle(
        location=[waypoints[i][0], waypoints[i][1]],
        radius=5,
        color='black',
        fill=True,
        fill_opacity=0.2
    ).add_to(m)

for index, row in gdf.iterrows():
    if isinstance(row['geometry'], Polygon):
        hull = folium.GeoJson(row['geometry'].__geo_interface__, name='convex_hull')
        Popup('wp' + str(row['label'])).add_to(hull)
        hull.add_to(m)

np.save(cfg.adjacency_matrix_save_dir, adjacency_matrix)
np.save(cfg.distance_matrix_save_dir, distance_matrix)
print('edge_list', len(edge_list))
folium.PolyLine(
    locations=edge_list,
    color="black",
    weight=5,
    opacity=0.7,
).add_to(m)
m